In [1]:

import matplotlib.pyplot as plt
import multiprocess as mp
import glob
import time
from tqdm import tqdm
import os
import sys

import dbfread 
import geopandas as gpd
import warnings
import matplotlib.pyplot as plt
import csv
import pyproj
from simpledbf import Dbf5
from datetime import datetime


### pick the data records with measurements of our interest

In [2]:
# import pandas as pd
# mesu = pd.read_excel('/home/opengeohub/xuemeng/work_xuemeng/spatiotemporal-soc-eu/NatbodDS_V6_EN/CH_SoilDataset_V6_EN.xlsx', sheet_name='7_Surv_Sample_Measur')
# cols = ['ID', 'COORDINATE_X_E', 'COORDINATE_Y_N', 'ANONYMIZATION', 'SOIL_SURVEY_DATE','DEPTH_FROM_cm','DEPTH_TO_cm']
# check = ['pH','Humus','Corg','Sand', 'Ton', 'Schluff','N','P','K','Kalk','P-AAE','K-AAE','RG']
# add = ['ANALYTICAL_PARAMETER','MEASURED_VALUE', 'UNIT', 'DETECTION_LIMIT','ANALYSIS_GROUP','SOIL_EXTRACTION_METHOD','SOIL_EXTRACTION_METHOD_REFERENCE',
#        'MEASURED_VALUE_LOWER_THAN_DETECTION_LIMIT', 'SOIL_ANALYSIS_METHOD','SOIL_ANALYSIS_METHOD_REFERENCE']

# mesuin = mesu.loc[mesu['ANALYTICAL_PARAMETER'].isin(check)][cols+add]
# mesuin = mesuin.dropna(subset=['MEASURED_VALUE'])

# # aggregate every column with regard to method for later use
# def concatenate_row_values(row):
#     return '--'.join(row[row.notna()])

# mtd_cols = ['ANALYSIS_GROUP','SOIL_ANALYSIS_METHOD','SOIL_ANALYSIS_METHOD_REFERENCE','SOIL_EXTRACTION_METHOD','SOIL_EXTRACTION_METHOD_REFERENCE']
# mesuin['method'] = mesuin[mtd_cols].apply(concatenate_row_values, axis=1)

# mesuin.to_csv('/home/opengeohub/xuemeng/work_xuemeng/spatiotemporal-soc-eu/NatbodDS_V6_EN/NABO_filtered.csv',index=False)

### organize the database into regular form, harmonize based on unit, method, etc.

In [1]:
import pandas as pd
import numpy as np
cols = ['ID', 'COORDINATE_X_E', 'COORDINATE_Y_N', 'ANONYMIZATION', 'SOIL_SURVEY_DATE','DEPTH_FROM_cm','DEPTH_TO_cm']

mesuin = pd.read_csv('/home/opengeohub/xuemeng/work_xuemeng/spatiotemporal-soc-eu/NatbodDS_V6_EN/NABO_filtered.csv')

para_list = []
info = ['MEASURED_VALUE','UNIT','method']

# create a function to examine the soil properties one by one
def exam_para(para_name,para_analy):
    temp = mesuin.loc[mesuin['ANALYTICAL_PARAMETER']==para_analy]
    temp = temp[cols+info]
    print(f'{para_name}-{para_analy} unit: ',temp['UNIT'].unique().tolist())

    infot = [f'{para_name}',f'{para_name}_unit',f'{para_name}_method']
    rename = dict(zip(info,infot))
    temp = temp.rename(columns=rename)

    no_nan_num = temp[para_name].notna().sum()
    method_record_num = temp[f'{para_name}_method'].notna().sum()
    
    print(f'{len(temp)} data records in total, {no_nan_num} with valid {para_name} values, {method_record_num} with measurement method recorded')
    temp_method = temp.loc[~temp[f'{para_name}_method'].isna()].reset_index(drop=True)

    # print('method numbers overview:')
    num = []
    name = []
    for i in temp_method[f'{para_name}_method'].unique().tolist():
        method_num = sum(sum([temp_method[f'{para_name}_method']==i]))
        name.append(i)
        num.append(method_num)
        # print(f'{i} - {method_num}')
    
    return tuple(zip(num,name)),temp

def print_overview(nnlist,filt=0):
    if filt==0:
        print('overview:')
        for i in range(len(nnlist)):
            print(f'{nnlist[i][1]} - {nnlist[i][0]}')
    else:
        print(f'overview with filter {filt}:')
        for i in range(len(nnlist)):
            if filt in nnlist[i][1]:
                print(f'{nnlist[i][1]} - {nnlist[i][0]}')

#### SOC

In [2]:
para_name = 'oc'
para_analy = 'Corg'
nnlist, temp = exam_para(para_name,para_analy)
print_overview(nnlist)
temp.loc[:,para_name] = temp[para_name]*10 # unit conversion

# drop data without method specified
temp = temp.loc[~temp[f'{para_name}_method'].isin(['BOKW','BOKW--Berechnung','BOKW--Titration--AgroLab--AgroLab','BOKW--Gemäss VSBo/VBBo--Gemäss VSBo/VBBo'])]

# convert oc to dru combustion method
convert_list = [i for i in temp[f'{para_name}_method'].unique().tolist() if 'CNS' not in i]
temp.loc[temp[f'{para_name}_method'].isin(convert_list),para_name] = temp.loc[temp[f'{para_name}_method'].isin(convert_list),para_name]*1.3
print()
print(f'{len(temp)} {para_name} values left')
para_list.append(temp)

oc-Corg unit:  ['%']
13309 data records in total, 13309 with valid oc values, 13309 with measurement method recorded
overview:
BOKW--Photometer--K2Cr2O7 - 49
BOKW - 1113
BOKW--Blacke&Walkly - 953
BOKW--Blacke_Walkly--K2Cr2O7 - 190
BOKW--Titration--K2Cr2O7--Corg - 199
BOKW--Titration--AgroLab--AgroLab - 40
BOKW--Gemäss VSBo/VBBo--Gemäss VSBo/VBBo - 32
BOKW--Berechnung - 1578
BOKW--Berechnung--K2Cr2O7 - 228
BOKW--Titration--Corg--K2Cr2O7--Corg - 3787
BOKW--Photometer--Corg--K2Cr2O7--Corg - 92
BOKW--CNS_Analyser--Verbrennung - 4696
BOKW--CNS_Analyser--ISO 10694 (1995)--HCl - 34
BOKW--CNS_Analyser--ISO 10694:1995--HCl--ISO 10694:1995 - 318

10546 oc values left


In [3]:
para_name = 'oc'
para_analy = 'Humus'
nnlist, temp = exam_para(para_name,para_analy)
print_overview(nnlist)
drop_list = [
    "BOKW--AAS--HCl+H2SO4",
    "BOKW--AAS--AgroLab--AgroLab",
    "BOKW--Berechnung",
    "BOKW--Berechnung_Humus_1.725",
    "BOKW--Berechnung_Humus_1.725--Corg--Corg",
    "BOKW--Berechnung--Corg",
    "BOKW--Gemäss VSBo/VBBo--Gemäss VSBo/VBBo",
    "BOKW--Pipettenmethode--KOM (Agroscope Band 1): KOM--H2O2_Dispergierung--KOM (Agroscope Band 1)",
    "BOKW--Titration",
    "BOKW"]

# drop data without method specified
temp = temp.loc[~temp[f'{para_name}_method'].isin(drop_list)]

# convert to dry combustion method
temp[para_name] = temp[para_name]*10/1.725 # unit conversion, hums-oc conversion
convert_list = [
    "BOKW--Blacke_Walkly--K2Cr2O7",
    "BOKW--Blacke_Walkly--K2Cr2O7--gemäss FAC",
    "BOKW--Titration--AC-Methode_C-1 [www.nabodat.ch]--K2Cr2O7--AC-Methode_C-1 [www.nabodat.ch]",
    "BOKW--Titration--vgl. AC-Methode_C-1--K2Cr2O7--vgl. AC-Methode_C-1",
    "BOKW--Titration--Referenzmethode_Corg--K2Cr2O7--Referenzmethode_Corg",
    "BOKW--Titration--Méthodes de référence des Stations de recherches Agroscope – code : Corg--K2Cr2O7--Méthodes de référence des Stations de recherches Agroscope – code : Corg",
    "BOKW--Titration--vgl. Referenzmethode_Corg--K2Cr2O7--vgl. Referenzmethode_Corg",
    "BOKW--Titration--Méthodes de référence des Stations de recherches Agroscope – code : Corg--HCl+H2SO4--Méthodes de référence des Stations de recherches Agroscope – code : Corg",
    "BOKW--Berechnung_Humus_1.725--K2Cr2O7",
    "BOKW--Berechnung--K2Cr2O7",
    "BOKW--Photometer--K2Cr2O7",
    "BOKW--Berechnung_Humus_1.725--Corg--K2Cr2O7--Corg",
    "BOKW--Titration--K2Cr2O7",
    "BOKW--Titration--K2Cr2O7--Corg",
    "BOKW--Titration--Corg--K2Cr2O7--Corg"
]
temp.loc[temp[f'{para_name}_method'].isin(convert_list),para_name] = temp.loc[temp[f'{para_name}_method'].isin(convert_list),para_name]*1.3

print()
print(f'{len(temp)} {para_name} values left')
para_list.append(temp)

oc-Humus unit:  ['%']
32400 data records in total, 32400 with valid oc values, 32400 with measurement method recorded
overview:
BOKW--Photometer--K2Cr2O7 - 348
BOKW - 7701
BOKW--Titration--AC-Methode_C-1 [www.nabodat.ch]--K2Cr2O7--AC-Methode_C-1 [www.nabodat.ch] - 7654
BOKW--Titration--Referenzmethode_Corg--K2Cr2O7--Referenzmethode_Corg - 13
BOKW--AAS--AgroLab--AgroLab - 959
BOKW--Berechnung_Humus_1.725--K2Cr2O7 - 190
BOKW--Berechnung_Humus_1.725 - 5087
BOKW--Gemäss VSBo/VBBo--Gemäss VSBo/VBBo - 325
BOKW--Berechnung - 379
BOKW--Berechnung--K2Cr2O7 - 44
BOKW--Titration--Méthodes de référence des Stations de recherches Agroscope – code : Corg--K2Cr2O7--Méthodes de référence des Stations de recherches Agroscope – code : Corg - 332
BOKW--Titration--Méthodes de référence des Stations de recherches Agroscope – code : Corg--HCl+H2SO4--Méthodes de référence des Stations de recherches Agroscope – code : Corg - 9
BOKW--Berechnung_Humus_1.725--Corg--K2Cr2O7--Corg - 583
BOKW--Titration--K2Cr2O7 - 

#### pH

In [4]:
# ph-h2o
para_name = 'ph_h2o'
para_analy = 'pH'
nnlist, temp = exam_para(para_name,para_analy)
print_overview(nnlist,filt='H2O')

# keep only ph measured with H2O
keep_list = []
for i in range(len(nnlist)):
    if 'H2O' in nnlist[i][1]:
        keep_list.append(nnlist[i][1])
temp = temp.loc[temp[f'{para_name}_method'].isin(keep_list)]

# drop data without method specified
drop_list = [
    "BOKW--Messelektrode--gemäss VSBo--H2O",
    "BOKW--Gemäss VSBo/VBBo--H2O--Gemäss VSBo/VBBo",
    "BOKW--Messelektrode--AgroLab--H2O--AgroLab",
    "BOKW--Messelektrode--pH--H2O--pH",
    "BOKW--Messelektrode--H2O",
    "BOKW--H2O--pH",
    "BOKW--H2O"
]
temp = temp.loc[~temp[f'{para_name}_method'].isin(drop_list)]

print()
print(f'{len(temp)} {para_name} values left')
para_list.append(temp)

ph_h2o-pH unit:  [' - ']
62410 data records in total, 62410 with valid ph_h2o values, 62410 with measurement method recorded
overview with filter H2O:
BOKW--Messelektrode--Schw. Referenzmethoden der Eidg. landw. Forschungsanstalten; Band 2; Code: pH--H2O--Schw. Referenzmethoden der Eidg. landw. Forschungsanstalten; Band 2; Code: pH - 935
BOKW--H2O - 2145
BOKW--Messelektrode--AC-Methode_HP-1-Teil1 [www.nabodat.ch]--H2O--AC-Methode_HP-1-Teil1 [www.nabodat.ch] - 4489
BOKW--FAC 3.1--H2O - 1705
BOKW--Messelektrode--AgroLab--H2O--AgroLab - 785
BOKW--Messelektrode--H2O - 2004
BOKW--Gemäss VSBo/VBBo--H2O--Gemäss VSBo/VBBo - 186
BOKW--Messelektrode--ISO 10523--H2O--FAC A.3.1 - 21
BOKW--Messelektrode--Méthodes de référence des Stations de recherches Agroscope – code : pH--H2O--Méthodes de référence des Stations de recherches Agroscope – code : pH - 429
BOKW--H2O--pH - 2309
BOKW--Messelektrode--pH--H2O--pH - 462
BOKW--Messelektrode--vgl. AC-Methode_HP-1-I--H2O--vgl. AC-Methode_HP-1-I - 64
BOKW--M

In [5]:
# ph-CaCl2
para_name = 'ph_cacl2'
para_analy = 'pH'
nnlist, temp = exam_para(para_name,para_analy)
print_overview(nnlist,filt='CaCl2')

# keep only ph measured with H2O
keep_list = []
for i in range(len(nnlist)):
    if 'CaCl2' in nnlist[i][1]:
        keep_list.append(nnlist[i][1])
temp = temp.loc[temp[f'{para_name}_method'].isin(keep_list)]

# drop data without method specified
drop_list = [
    "BOKW--Gemäss VSBo/VBBo--CaCl2--Gemäss VSBo/VBBo",
    "BOKW--Messelektrode--pH--CaCl2--pH",
    "BOKW--Messelektrode--AgroLab--CaCl2--AgroLab",
    "BOKW--Messelektrode--CaCl2",
    "BOKW--CaCl2"
]
temp = temp.loc[~temp[f'{para_name}_method'].isin(drop_list)]

print()
print(f'{len(temp)} {para_name} values left')
para_list.append(temp)

ph_cacl2-pH unit:  [' - ']
62410 data records in total, 62410 with valid ph_cacl2 values, 62410 with measurement method recorded
overview with filter CaCl2:
BOKW--Messelektrode--Schw. Referenzmethoden der Eidg. landw. Forschungsanstalten; Band 2; Code: pH-C--CaCl2--Schw. Referenzmethoden der Eidg. landw. Forschungsanstalten; Band 2; Code: pH-C - 2539
BOKW--CaCl2 - 2800
BOKW--Messelektrode--AC-Methode_HP-2 [www.nabodat.ch]--CaCl2--AC-Methode_HP-2 [www.nabodat.ch] - 12238
BOKW--FAC 3.3--CaCl2 - 2432
BOKW--Messelektrode--AgroLab--CaCl2--AgroLab - 1286
BOKW--Messelektrode--CaCl2 - 2079
BOKW--Messelektrode--pH-C--CaCl2 - 199
BOKW--Gemäss VSBo/VBBo--CaCl2--Gemäss VSBo/VBBo - 229
BOKW--Messelektrode--CaCl2--FAC, 3.3 - 2205
BOKW--Messelektrode--ISO 10523--CaCl2--FAC A. 3.3 - 46
BOKW--Messelektrode--CaCl2--gemäss FAC - 62
BOKW--Messelektrode--CaCl2--VDLUFA A5.1.1 - 343
BOKW--Messelektrode--pH-C--CaCl2--pH-C - 4342
BOKW--Messelektrode--VDLUFA A5.1.1--CaCl2--VDLUFA A5.1.1 - 35
BOKW--Messelektrode

In [6]:
# # ph-kcl, drop all since no explicit methods recorded
# para_name = 'ph_kcl'
# para_analy = 'pH'
# nnlist, temp = exam_para(para_name,para_analy)
# print_overview(nnlist,filt='KCl')

# # keep only ph measured with H2O
# keep_list = []
# for i in range(len(nnlist)):
#     if 'KCl' in nnlist[i][1]:
#         keep_list.append(nnlist[i][1])
# temp = temp.loc[temp[f'{para_name}_method'].isin(keep_list)]
# print()
# print(f'{len(temp)} {para_name} values left')
# para_list.append(temp)

### N

In [7]:
para_name = 'N'
para_analy = 'N'
nnlist, temp = exam_para(para_name,para_analy)
print_overview(nnlist)

temp[para_name] = temp[para_name]/1000 # mg/kg -> g/kg

drop_list = ['NAEHR','NAEHR--AgroLab--AgroLab']
temp = temp.loc[~temp[f'{para_name}_method'].isin(drop_list)]
print()
print(f'{len(temp)} {para_name} values left')
para_list.append(temp)

N-N unit:  ['mg/kg']
6108 data records in total, 6108 with valid N values, 6108 with measurement method recorded
overview:
NAEHR--Kjeldahl - 289
NAEHR--AgroLab--AgroLab - 549
NAEHR--CNS_Analyser--Verbrennung - 4466
NAEHR--CNS_Analyser--DIN EN 16168 - 46
NAEHR--Kjeldahl--automatische Coloriemetrie (SKALAR) - 56
NAEHR--Kjeldahl--VDLUFA_A2_2_1 - 343
NAEHR--Kjeldahl--HR-N-KJ--VDLUFA_A2_2_1--HR-N-KJ - 35
NAEHR - 6
NAEHR--Kjeldahl--ISO 11261:1995--H2SO4+Salicylsäure+Na2S2O3--ISO 11261:1995 - 318

5553 N values left


### CaCO3

In [8]:
para_name = 'caco3'
para_analy = 'Kalk'
nnlist, temp = exam_para(para_name,para_analy)
print_overview(nnlist)

temp[para_name] = temp[para_name]*10 # % -> g/kg

drop_list = [
    "BOKW--Gemäss VSBo/VBBo--Gemäss VSBo/VBBo",
    "BOKW--Titration--AgroLab--AgroLab",
    "BOKW--HCl--CaCO3",
    "BOKW--CaCO3--HCl--CaCO3",
    "BOKW--Berechnung--CaCO3--HCl--CaCO3",
    "BOKW--HCl",
    "BOKW"
]
temp = temp.loc[~temp[f'{para_name}_method'].isin(drop_list)]

print()
print(f'{len(temp)} {para_name} values left')
para_list.append(temp)

caco3-Kalk unit:  ['%']
23108 data records in total, 23108 with valid caco3 values, 23108 with measurement method recorded
overview:
BOKW--Passon--Schw. Referenzmethoden der Eidg. landw. Forschungsanstalten; Band 2; Code: CaCO3--HCl--Schw. Referenzmethoden der Eidg. landw. Forschungsanstalten; Band 2; Code: CaCO3 - 350
BOKW - 2023
BOKW--Scheibler--AC-Methode_Ca-2 [www.nabodat.ch]--HCl--AC-Methode_Ca-2 [www.nabodat.ch] - 9483
BOKW--Scheibler--Referenzmethode_CaCO3--HCl--Referenzmethode_CaCO3 - 8
BOKW--FAC 9.1 - 369
BOKW--Titration--AgroLab--AgroLab - 1014
BOKW--Passon - 40
BOKW--Gemäss VSBo/VBBo--Gemäss VSBo/VBBo - 247
BOKW--Passon--HCl--FAC, A.9.1 - 1518
BOKW--Passon--FAC A.9.1 - 46
BOKW--HCl--CaCO3 - 2668
BOKW--Scheibler--HCl - 1
BOKW--Extraktion gem FAC S. 109ff. Bestimmungsprinzip: Volumetrische Dossierung auf PASSON- Apparat - 56
BOKW--Passon--HCl--CaCO3 - 343
BOKW--Passon--CaCO3--HCl--CaCO3 - 35
BOKW--HCl - 481
BOKW--Scheibler--vgl. Referenzmethode_CaCO3--HCl--vgl. Referenzmethode

### texture

In [9]:
para_name = 'clay'
para_analy = 'Ton'
nnlist, temp = exam_para(para_name,para_analy)
print_overview(nnlist)

drop_list = [
    "BOKW--Gemäss VSBo/VBBo--Gemäss VSBo/VBBo",
    "BOKW--Berechnung--KOM--H2O2_Dispergierung--KOM",
    "BOKW--Berechnung--KO2--H2O2_Dispergierung",
    "BOKW--Berechnung--H2O2_Dispergierung",
    "BOKW--Berechnung",
    "BOKW--Berechnung--Dispersion",
    "BOKW--Berechnung--Suspension",
    "BOKW--Suspension",
    "BOKW"
]
temp = temp.loc[~temp[f'{para_name}_method'].isin(drop_list)]

print()
print(f'{len(temp)} {para_name} values left')
para_list.append(temp)

clay-Ton unit:  ['%']
41069 data records in total, 41069 with valid clay values, 41069 with measurement method recorded
overview:
BOKW - 13127
BOKW--Pipettenmethode--AC-Methode_KO-2 [www.nabodat.ch]--H2O2_Dispergierung--AC-Methode_KO-2 [www.nabodat.ch] - 9169
BOKW--Pipettenmethode--Referenzmethode_KOM--H2O2_Dispergierung--Referenzmethode_KOM - 12
BOKW--FAC 16.1 - 1002
BOKW--Pipettenmethode - 2535
BOKW--Pipettenmethode--KOM--Dispersion--KOM - 519
BOKW--Gemäss VSBo/VBBo--Gemäss VSBo/VBBo - 253
BOKW--Messelektrode--Dispersion - 26
BOKW--Pipettenmethode--Dispersion - 17
BOKW--Pipettenmethode--Méthodes de référence des Stations de recherches Agroscope – code : KOM--H2O2_Dispergierung--Méthodes de référence des Stations de recherches Agroscope – code : KOM - 456
BOKW--Pipettenmethode--KOM--H2O2_Dispergierung--KOM - 4470
BOKW--Messelektrode--CaCl2 - 2
BOKW--Pipettenmethode--H2O2_Dispergierung--gemäss FAP - 55
BOKW--Berechnung--Suspension - 133
BOKW--Pipettenmethode--Dispersion--KOF - 343
BOKW

In [10]:
para_name = 'sand'
para_analy = 'Sand'
nnlist, temp = exam_para(para_name,para_analy)
print_overview(nnlist)

drop_list = [
    "BOKW--Gemäss VSBo/VBBo--Gemäss VSBo/VBBo",
    "BOKW--Berechnung--KOM--H2O2_Dispergierung--KOM",
    "BOKW--Berechnung--KO2--H2O2_Dispergierung",
    "BOKW--Berechnung--H2O2_Dispergierung",
    "BOKW--Berechnung",
    "BOKW--Berechnung--Dispersion",
    "BOKW--Berechnung--Suspension",
    "BOKW--Suspension",
    "BOKW"
]
temp = temp.loc[~temp[f'{para_name}_method'].isin(drop_list)]

print()
print(f'{len(temp)} {para_name} values left')
para_list.append(temp)

sand-Sand unit:  ['%']
32137 data records in total, 32137 with valid sand values, 32137 with measurement method recorded
overview:
BOKW - 11441
BOKW--sieben_wiegen--AC-Methode_KO-3 [www.nabodat.ch]--H2O2_Dispergierung--AC-Methode_KO-2 [www.nabodat.ch] - 9154
BOKW--Pipettenmethode--Referenzmethode_KOM--H2O2_Dispergierung--Referenzmethode_KOM - 12
BOKW--FAC 16.1 - 1002
BOKW--Pipettenmethode - 1033
BOKW--Pipettenmethode--KOM--Dispersion--KOM - 519
BOKW--Gemäss VSBo/VBBo--Gemäss VSBo/VBBo - 253
BOKW--Berechnung - 511
BOKW--Berechnung--Dispersion - 26
BOKW--Pipettenmethode--Méthodes de référence des Stations de recherches Agroscope – code : KOM--H2O2_Dispergierung--Méthodes de référence des Stations de recherches Agroscope – code : KOM - 458
BOKW--Pipettenmethode--KOM--H2O2_Dispergierung--KOM - 4322
BOKW--Pipettenmethode--Dispersion - 4
BOKW--Pipettenmethode--H2O2_Dispergierung--gemäss FAP - 55
BOKW--Berechnung--Suspension - 133
BOKW--Pipettenmethode--Dispersion--KOF - 343
BOKW--Pipettenmet

In [11]:
para_name = 'silt'
para_analy = 'Schluff'
nnlist, temp = exam_para(para_name,para_analy)
print_overview(nnlist)

drop_list = [
    "BOKW--Gemäss VSBo/VBBo--Gemäss VSBo/VBBo",
    "BOKW--Messelektrode--Dispersion",
    "BOKW--Berechnung--Suspension",
    "BOKW--Suspension",
    "BOKW"
]
temp = temp.loc[~temp[f'{para_name}_method'].isin(drop_list)]

print()
print(f'{len(temp)} {para_name} values left')
para_list.append(temp)

silt-Schluff unit:  ['%']
39439 data records in total, 39439 with valid silt values, 39439 with measurement method recorded
overview:
BOKW - 11809
BOKW--Pipettenmethode--AC-Methode_KO-2 [www.nabodat.ch]--H2O2_Dispergierung--AC-Methode_KO-2 [www.nabodat.ch] - 9171
BOKW--Pipettenmethode--Referenzmethode_KOM--H2O2_Dispergierung--Referenzmethode_KOM - 12
BOKW--FAC 16.1 - 1002
BOKW--Pipettenmethode - 2566
BOKW--Pipettenmethode--KOM--Dispersion--KOM - 199
BOKW--Gemäss VSBo/VBBo--Gemäss VSBo/VBBo - 253
BOKW--Messelektrode--Dispersion - 26
BOKW--Pipettenmethode--Dispersion - 19
BOKW--Pipettenmethode--Méthodes de référence des Stations de recherches Agroscope – code : KOM--H2O2_Dispergierung--Méthodes de référence des Stations de recherches Agroscope – code : KOM - 460
BOKW--Pipettenmethode--KOM--H2O2_Dispergierung--KOM - 4467
BOKW--Pipettenmethode--H2O2_Dispergierung--gemäss FAP - 55
BOKW--Berechnung--Suspension - 133
BOKW--Pipettenmethode--Dispersion--KOF - 343
BOKW--Pipettenmethode--KOF--Dis

### P, phosphorus

In [12]:
para_name = 'P'
para_analy = 'P-AAE'
nnlist, temp1 = exam_para(para_name,para_analy)
# print_overview(nnlist)

para_name = 'P'
para_analy = 'P'
nnlist, temp2 = exam_para(para_name,para_analy)
# print_overview(nnlist)

temp = pd.concat([temp1,temp2])
print(f'{len(temp)} data in total')

keep_list = ['NAEHR--Photometer--ISO 11263:1994--NaHCO3+NaOH+C--ISO 11263:1994']
temp = temp.loc[temp[f'{para_name}_method'].isin(keep_list)]

print()
print(f'{len(temp)} {para_name} values left')
para_list.append(temp)

P-P-AAE unit:  ['mg/kg']
402 data records in total, 402 with valid P values, 402 with measurement method recorded
P-P unit:  ['mg/kg']
8947 data records in total, 8947 with valid P values, 8947 with measurement method recorded
9349 data in total

318 P values left


### K, potassium

In [13]:
para_name = 'K'
para_analy = 'K-AAE'
nnlist, temp1 = exam_para(para_name,para_analy)
# print_overview(nnlist)

para_name = 'K'
para_analy = 'K'
nnlist, temp2 = exam_para(para_name,para_analy)
# print_overview(nnlist)

temp = pd.concat([temp1,temp2])
print(f'{len(temp)} data in total')

drop_list = [
    "NAEHR--AAS--DIN 38406--HNO3+HCl--FAL AD-KW-Ex",
    "NAEHR--AAS-F--Flusssaeure_Gemisch--GLA_Geologisches Landesamt Bayern_Fachbericht 2",
    "NAEHR--Gemäss VSBo/VBBo--Gemäss VSBo/VBBo",
    "NAEHR--AAS-F--HNO3+HCl",
    "NAEHR--ICP-MS--HNO3+HCl",
    "NAEHR--ICP-MS--MMI",
    "NAEHR--XRF"
]
temp = temp.loc[~temp[f'{para_name}_method'].isin(drop_list)]


print()
print(f'{len(temp)} {para_name} values left')
para_list.append(temp)

K-K-AAE unit:  ['mg/kg']
402 data records in total, 402 with valid K values, 402 with measurement method recorded
K-K unit:  ['mg/kg']
5456 data records in total, 5456 with valid K values, 5456 with measurement method recorded
5858 data in total

3610 K values left


### bulk density

In [15]:
para_name = 'bulk_density'
para_analy = 'RG'
nnlist, temp = exam_para(para_name,para_analy)
print_overview(nnlist)

drop_list = ['PHYS--Gemäss VSBo/VBBo--Gemäss VSBo/VBBo','PHYS']
temp = temp.loc[~temp[f'{para_name}_method'].isin(drop_list)]

print()
print(f'{len(temp)} {para_name} values left')
para_list.append(temp)

bulk_density-RG unit:  ['g/cm3']
2431 data records in total, 2431 with valid bulk_density values, 2431 with measurement method recorded
overview:
BOKW--Berechnung--Schw. Referenzmethoden der Eidg. landw. Forschungsanstalten; Band 2; Code: PYZYL-PA-- kein Aufschluss - 123
PHYS - 1671
PHYS--Desorption-Messystem - 457
PHYS--Gemäss VSBo/VBBo--Gemäss VSBo/VBBo - 116
PHYS--Desorption-Messystem--praeparieren_konditionieren--Referenzmethode Agroscope - 30
PHYS--Wiegen--vgl. Referenzmethode_PYZYL-D--vgl. Referenzmethode_PYZYL-PA - 34

644 bulk_density values left


## merge all the properties together

In [16]:
final = pd.concat(para_list)
ulist = [i for i in final.columns.tolist() if 'unit' in i or 'method' in i]
final = final.drop(columns=ulist)
final

,ID,COORDINATE_X_E,COORDINATE_Y_N,ANONYMIZATION,SOIL_SURVEY_DATE,DEPTH_FROM_cm,DEPTH_TO_cm,oc,ph_h2o,ph_cacl2,N,caco3,clay,sand,silt,P,K,bulk_density
1694,212,2615525,1265719,0,30.06.1984,0.0,18.0,50.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1696,212,2615525,1265719,0,30.06.1984,18.0,55.0,23.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1699,213,2629070,1257753,0,01.01.1984,0.0,15.0,102.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1746,332,2617654,1262648,0,01.01.1984,0.0,10.0,57.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1748,332,2617654,1262648,0,01.01.1984,11.0,20.0,40.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
229127,31843,2629001,1253000,0,04.12.1991,0.0,20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.20
229143,31847,2631001,1253100,0,04.12.1991,0.0,20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.38
229153,31849,2633001,1257000,0,03.12.1991,0.0,20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.33
229162,31851,2609000,1259000,0,11.12.1991,0.0,20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.44


In [17]:
merge = (final.groupby(cols)
            .agg({
                'ph_h2o': 'mean',
                'ph_cacl2': 'mean',
                # 'ph_KCl': 'mean',
                'oc': 'mean',
                'sand': 'mean',
                'clay': 'mean',
                'silt': 'mean',
                'N': 'mean',
                'P': 'mean',
                'K': 'mean',
                'caco3': 'mean',
                'bulk_density':'mean'
                # 'ph_H2O_SOIL_ANALYSIS_METHOD_REFERENCE': 'first',
                # 'ph_CaCl2_SOIL_ANALYSIS_METHOD_REFERENCE': 'first',
                # 'ph_KCl_SOIL_ANALYSIS_METHOD_REFERENCE': 'first',
                # 'oc_SOIL_ANALYSIS_METHOD_REFERENCE': 'first',
                # 'Sand_SOIL_ANALYSIS_METHOD_REFERENCE': 'first',
                # 'Ton_SOIL_ANALYSIS_METHOD_REFERENCE': 'first',
                # 'Schluff_SOIL_ANALYSIS_METHOD_REFERENCE': 'first',
                # 'N_SOIL_ANALYSIS_METHOD_REFERENCE': 'first',
                # 'P_SOIL_ANALYSIS_METHOD_REFERENCE': 'first',
                # 'K_SOIL_ANALYSIS_METHOD_REFERENCE': 'first',
                # 'Kalk_SOIL_ANALYSIS_METHOD_REFERENCE': 'first'
            })
            .reset_index())

name1 = ['COORDINATE_X_E', 'COORDINATE_Y_N', 'ANONYMIZATION','SOIL_SURVEY_DATE', 'DEPTH_FROM_cm', 'DEPTH_TO_cm']
name2 = ['lon', 'lat', 'anonymization','date', 'hzn_top', 'hzn_btm']

rename = dict(zip(name1,name2))
merge = merge.rename(columns=rename)

merge

,ID,lon,lat,anonymization,date,hzn_top,hzn_btm,ph_h2o,ph_cacl2,oc,sand,clay,silt,N,P,K,caco3,bulk_density
0,4,2632671,1263170,0,12.07.1993,0.0,20.0,7.5,7.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4,2632671,1263170,0,12.10.1992,0.0,20.0,7.8,7.300000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4,2632671,1263170,0,12.10.1992,20.0,40.0,7.9,7.300000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,2632671,1263170,0,12.10.1992,40.0,80.0,8.2,7.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,2612560,1258600,0,01.01.1987,0.0,20.0,7.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41327,34406,2709000,1173000,1,16.03.2017,0.0,20.0,NaN,5.966667,39.160000,NaN,NaN,NaN,3.946667,NaN,NaN,NaN,NaN
41328,34406,2709000,1173000,1,17.04.2013,0.0,20.0,NaN,5.433333,39.266667,NaN,NaN,NaN,4.433333,NaN,NaN,0.0,NaN
41329,34406,2709000,1173000,1,19.03.2020,0.0,20.0,NaN,5.900000,37.010000,NaN,NaN,NaN,3.843667,NaN,NaN,NaN,NaN
41330,34406,2709000,1173000,1,25.03.2015,0.0,20.0,NaN,5.300000,34.033333,NaN,NaN,NaN,3.400000,NaN,NaN,0.0,NaN


In [18]:
merge.to_csv('/home/opengeohub/xuemeng/work_xuemeng/spatiotemporal-soc-eu/NatbodDS_V6_EN/swiss_harmonized_v1.csv',index=False)

### cordinate & time issues

In [23]:
merge = merge.loc[merge['anonymization']==0] # drop all the columns without explicit coordinates
merge['time'] = merge['date'].str[-4::].astype(float)
merge = merge.loc[merge['time']>=2000] # drop all the columns without explicit coordinates

merge

/tmp/ipykernel_1060/3575463034.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merge['time'] = merge['date'].str[-4::].astype(float)


,ID,lon,lat,anonymization,date,hzn_top,hzn_btm,ph_h2o,ph_cacl2,oc,sand,clay,silt,N,P,K,caco3,bulk_density,time
202,61,2626491,1253300,0,05.12.2008,0.0,20.0,6.51,5.980,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008.0
208,61,2626491,1253300,0,21.11.2003,0.0,20.0,NaN,5.875,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2003.0
209,62,2626938,1247875,0,08.12.2008,0.0,20.0,6.96,6.660,56.521739,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008.0
214,62,2626938,1247875,0,26.11.2003,0.0,20.0,NaN,6.050,52.753623,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2003.0
215,63,2608370,1267121,0,06.11.2008,0.0,20.0,7.50,7.180,40.695652,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41061,34231,2603702,1111815,0,22.09.2008,0.0,10.0,NaN,NaN,66.000000,NaN,10.1,NaN,NaN,NaN,NaN,NaN,NaN,2008.0
41062,34232,2655607,1140358,0,23.09.2008,0.0,10.0,NaN,NaN,59.000000,NaN,12.3,NaN,NaN,NaN,NaN,NaN,NaN,2008.0
41063,34233,2741600,1234858,0,02.07.2008,0.0,10.0,NaN,NaN,49.000000,NaN,8.4,NaN,NaN,NaN,NaN,NaN,NaN,2008.0
41064,34234,2639721,1163225,0,23.09.2008,0.0,10.0,NaN,NaN,110.000000,NaN,11.2,NaN,NaN,NaN,NaN,NaN,NaN,2008.0


In [24]:
import pyproj

# Define the coordinate systems
ch1903 = pyproj.CRS('EPSG:21781')  # CH1903/LV03
ch1903plus = pyproj.CRS('EPSG:2056')  # CH1903+/LV95
wgs84 = pyproj.CRS('EPSG:4326')  # GPS coordinates

# Create a transformer
transformer_to_wgs84_from_ch1903 = pyproj.Transformer.from_crs(ch1903, wgs84, always_xy=True)
transformer_to_wgs84_from_ch1903plus = pyproj.Transformer.from_crs(ch1903plus, wgs84, always_xy=True)

# Example conversion from CH1903
lat, lon = transformer_to_wgs84_from_ch1903.transform(easting, northing)

# Example conversion from CH1903+
lat, lon = transformer_to_wgs84_from_ch1903plus.transform(easting, northing)


NameError: name 'easting' is not defined

In [ ]:
final